# 스파크 스트리밍 실습 1교시 : 스파크 스트리밍 소개

> 구조화된 스파크 스트리밍 애플리케이션은 크게 5단계로 구분하여 설계합니다 (스키마/소스읽기, 변환코드작성, 싱크코드작성, 트리거코드작성, 애플리케이션기동), 이번 과정에는 JSON 파일을 읽어서 스트리밍 소스로 활용하는 가장 간단한 예제를 통해서 스파크 스트리밍의 동작방식에 대해 이해합니다

## 학습 목표

* 스파크 스트리밍 애플리케이션을 구현하는 5가지 단계를 이해합니다
  - 스키마/소스 읽기
  - 변환 코드 작성
  - 싱크 코드 작성
  - 트리거 코드 작성
  - 애플리케이션 기동 및 종료
* 특정 경로에 존재하는 JSON 파일을 파일 단위로 읽는 스트리밍 소스를 생성합니다
* 특정 컬럼을 기준으로 집계함수를 적용한 변환 로직을 생성합니다
* 콘솔에 결과를 출력하는 싱크를 구성하여 스트리밍 애플리케이션을 시작합니다
* 약 1분간 스트리밍 처리를 수행하고, 애플리케이션을 종료합니다


## 목차
* [1. 소스 스키마 읽기](#1.-소스-스키마-읽기)
* [2. 소스 읽기 코드 작성](#2.-소스-읽기-코드-작성)
* [3. 변환 코드 작성](#3.-변환-코드-작성)
* [4. 싱크 코드 작성](#4.-싱크-코드-작성)
* [5. 트리거 코드 작성](#5.-트리거-코드-작성)
* [6. 애플리케이션 기동 및 종료](#6.-애플리케이션-기동-및-종료)


In [57]:
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *
from IPython.display import display, display_pretty, clear_output, JSON

spark = (
    SparkSession
    .builder
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size

# 공통 데이터 위치
home_jovyan = "/home/jovyan"
work_data = f"{home_jovyan}/work/data"
work_dir=!pwd
work_dir = work_dir[0]

# 로컬 환경 최적화
spark.conf.set("spark.sql.shuffle.partitions", 5) # the number of partitions to use when shuffling data for joins or aggregations.
spark.conf.set("spark.sql.streaming.forceDeleteTempCheckpointLocation", "true")

## 1. 소스 스키마 읽기

In [58]:
# 소스 스키마
activityPath = f"{work_data}/activity-data"
activityData = spark.read.option("inferSchema", "true").json(activityPath)
display(activityData.limit(2))
activityData.printSchema()

Arrival_Time,Creation_Time,Device,Index,Model,User,gt,x,y,z
1424686735090,1424686733090638193,nexus4_1,18,nexus4,g,stand,3.356934E-4,-5.645752E-4,-0.018814087
1424686735292,1424688581345918092,nexus4_2,66,nexus4,g,stand,-0.005722046,0.029083252,0.005569458


root
 |-- Arrival_Time: long (nullable = true)
 |-- Creation_Time: long (nullable = true)
 |-- Device: string (nullable = true)
 |-- Index: long (nullable = true)
 |-- Model: string (nullable = true)
 |-- User: string (nullable = true)
 |-- gt: string (nullable = true)
 |-- x: double (nullable = true)
 |-- y: double (nullable = true)
 |-- z: double (nullable = true)



## 2. 소스 읽기 코드 작성

> 예제에서는 json 파일을 소스로 사용했으며, `format("json").load(path)` 대신 `json(path)` 를 사용해도 동일합니다

#### [Input Sources](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#input-sources)
| 포맷 | 특징 | 옵션 |
| --- | --- | --- |
| File Source | parquet, json, csv 등의 파일의 읽기 | .option("path", "path/to/read"), 콤마 구분자로 `여러 경로를 지정할 수 없습니다` |
| Kafka Source | kafka 읽기를 위한 소스 | .option("kafka.bootstrap.servers", "host1:port1,host2:port2") 옵션을 통해 카프카 클러스터를 지정하고, .option("topic", "updates") 옵션을 통해 토픽을 지정합니다 |
| Socket Source | 소켓 서버로부터 스트리밍을 읽기 위한 소스 | .option("host", "localhost"), .option("port", 9999) 옵션을 통해 소켓서버를 지정합니다, includeTimestamp 옵션으로 `value`, `timestamp` 지원 |
| Rate Source | 트리밍 디버깅을 위한 포맷이며, 콘솔로 바로 출력합니다 | Debug Only |

* File Source Options
  - maxFilesPerTrigger : maximum number of new files to be considered in every trigger
  - latestFirst : whether to process the latest new files first, useful when there is a large backlog of files (default: false)
  - maxFileAge : A mapping from a file that we have processed to some timestamp it was last modified


In [59]:
# 소스 읽기
activitySchema = activityData.schema
activityReader = (
    spark
    .readStream
    .schema(activitySchema)
    .format("json")
    .option("maxFilesPerTrigger", 1)
    .load(activityPath)
)

## 3. 변환 코드 작성

In [60]:
# 데이터 변환
activityCounter = activityReader.groupBy("gt").count()

## 4. 싱크 코드 작성

> 예제에서는 디버깅을 위한 console 포맷과 update 모드를 사용했습니다

#### [Output Sinks](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#output-sinks)
| 포맷 | 특징 | 옵션 |
| --- | --- | --- |
| File Sink | parquet, json, csv 등의 파일을 저장 | .option("path", "path/to/write") |
| Kafka Sink | kafka 저장을 위한 싱크 | .option("kafka.bootstrap.servers", "host1:port1,host2:port2") 옵션을 통해 카프카 클러스터를 지정하고, .option("topic", "updates") 옵션을 통해 토픽을 지정합니다 |
| Foreach Sink | 스트리밍 싱크를 지원하지 않는 경우에도 임의의 연산을 수행을 통해 저장할 수 있는 옵션 | - |
| Console Sink | 트리밍 디버깅을 위한 포맷이며, 콘솔로 바로 출력합니다 | Debug Only |
| Memory Sink | 스트리밍 디버깅을 위한 포매싱며, 임의의 테이블로 출력합니다 | Debug Only, .queryName("tableName") 옵션을 통해 테이블 이름을 지정합니다 |


#### [Output Modes](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#output-modes)

| 출력 모드 | 특징 | 옵션 |
| --- | --- | --- |
| Append Mode | 신규로 발생한 로우의 추가만 가능한 모드이며, 기존 결과 테이블에 저장된 로우들은 변경되지 않습니다 | .option("path", "path/to/file") |
| Update Mode | 마지막 트리거 이후에 발생한 모든 로우가 외부 저장소에 변경되는 모드 | MySQL 과 같이 업데이트를 지원하는 싱크 저장소만 사용할 수 있으며, 쿼리가 Aggregation 을 포함하지 않는다면 Complete 와 동일하게 동작합니다 |
| Complete Mode | 매번 결과 테이블 전체를 갱신하는 모드이며 | 모든 로우를 매번 출력할 만큼 충분히 결과가 작은 경우에 사용할 수 있습니다 |

In [61]:
# 싱크 쓰기
queryName = "activity_count"
checkpointLocation = f"{work_dir}/tmp/{queryName}"
activityWriter = (
    activityCounter
    .writeStream
    .queryName(queryName)
    .format("console")
    .outputMode("complete")
)

## 5. 트리거 코드 작성

#### [Triggers](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#triggers)

| 트리거 유형 | 특징 | 옵션 |
| --- | --- | --- |
| Unspecified (Default) | 지정하지 않은 경우는 마이크로 배치 모드로 동작하며, 이전 마이크로 배치가 종료하는 즉시 다음 작업이 트리거 됩니다. | - |
| Fixed interval micro-batches | 제공되는 인터벌에 따라 동작하지만, 이전 작업이 인터벌 보다 늦게 종료되는 경우, 종료 즉시 다음 작업이 트리거 됩니다 | 새로운 작업이 없는 경우 트리거링 되지 않습니다 |
| One-time micro-batch | 처음 한 번만 수행됩니다 | 스케줄러 등을 통해 주기적으로 스트리밍 서비스를 수행할 때에 유용합니다 |
| Continuous with fixed checkpoint interval (experimental) | 마이크로 배치가 아니라 Continuous 모드로 동작합니다 | [Continuous Processing](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#continuous-processing) |

In [63]:
# 트리거
activityTrigger = (
    activityWriter
    .trigger(processingTime="1 second")
    .option("checkpointLocation", checkpointLocation)
)

## 6. 애플리케이션 기동 및 종료

* 기동 작업을 재수행 한다고 하더라도, 과거에 체크포인트에 저장된 값을 통해 기동되기 때문에 변화가 없습니다
* 다시 재기동 하기 위해서는 체크포인트를 삭제하고, 처음부터 기동하여야 합니다
```bash
# 임시파일 삭제
!rm -rf $checkpointLocation
```

In [64]:
# 기동
activityQuery = activityTrigger.start()
activityQuery.awaitTermination(30) # 노트북 특성상, 대기하면 다음 실행을 할 수 없으므로 30초만 수행하고 종료합니다
activityQuery.stop() # 특정 타스크가 처리 중에 timeout 되는 경우, 해당 스레드가 종료되어 예외가 발생할 수 있습니다

<br>

### 파일을 통한 스트리밍 시에 유의해야 할 특징 4가지
* 하나. 모든 파일들의 포맷과 스키마는 동일해야 하며, 그렇지 않은 경우 쿼리의 실패로 이어지게 됩니다
* 둘. 디렉토리 내의 파일 단위로 원자적으로 동작해야 하며, 한번에 정상적으로 읽어져야 합니다
  - 즉, append 되고 있는 파일의 경우 문제가 될 수 있으며, update 혹은 append 되어서는 안 됩니다
* 셋. 디렉토리 내의 새로운 파일들이 존재하더라도 모든 파일이 동시에 수행되지 않습니다
  - 동시 수행 임계치에 따라 병렬로 수행되며, 정해진 순서를 보장하지 않습니다.
  - maxFilesPerTrigger 옵션을 통해 처리율을 조정할 수 있습니다 - [프로그래밍 가이드](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#creating-streaming-dataframes-and-streaming-datasets)를 참고할 수 있습니다
* 넷. 파일 쓰기는 append 모드만 지원하며, 이미 존재하는 파일에는 append 될 수 없습니다
  - *end-to-end exatly-once gurantees* 지원은 _spark_metadata 경로에 존재하는 log 파일들을 통해 관리되어 가능하며, 종복 혹은 일부만 읽혀지는 경우는 없습니다. 
  - 만일 애플리케이션 재실행 시에 변경된 스키마로 데이터가 추가 되었다면, 쿼리 시에 해당 스키마가 조정되어야만 합니다
  
<br>

### <font color=blue>1. [중급]</font> 현재 생성된 스트리밍 애플리케이션을 처음부터 다시 수행해 보세요

* 처음부터 다시 스트리밍 처리를 하기 위해서는 어떻게 해야하나요?
  - 리눅스 삭제 명령 : `!rm -rf /path/to/remove`
  - 체크포인트 위치 : /home/jovyan/work/lgde-spark-stream/tmp/activity_count
  - 스트리밍 쓰기 객체 activityTrigger 는 실습 과정에서 정상적으로 생성되었다고 가정합니다

* 현재 activityQuery 를 다시 수행하되 충분히 긴 시간동안 수행하여 오류 메시지가 발생하지 않도록 수행해 보세요
  - 타임아웃 : 80초 이상
  - 쿼리 실행 이후 애플리케이션을 반드시 종료해 주세요

<details><summary>[정답] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
! rm -rf /home/jovyan/work/lgde-spark-stream/tmp/activity_count
activityQuery = activityTrigger.start()
activityQuery.awaitTermination(80)
activityQuery.stop()
```

</details>

In [37]:
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
! rm -rf /home/jovyan/work/lgde-spark-stream/tmp/activity_count
activityQuery = activityTrigger.start()
activityQuery.awaitTermination(80)
activityQuery.stop()

### <font color=red>2. [고급]</font> 스키마를 읽지 않고 직접 지정하는 코드를 작성해 보세요

* [pyspark.sql.types.StructType](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.sql.types.StructType.html?highlight=structtype#pyspark.sql.types.StructType) 참고
  - `customSchema = StructType([StructField("f1", StringType(), True)])` 와 같은 형식으로 작성합니다

* 기존에 작성된 코드를 활용하여 하나의 Cell 안에 작성해 보세요
  - 타임아웃 : 10초 내외
  - 쿼리 실행 이후 애플리케이션을 반드시 종료해 주세요

<details><summary>[정답] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
customSchema = (
    StructType()
    .add(StructField("Arrival_Time", LongType()))
    .add(StructField("Creation_Time", LongType()))
    .add(StructField("Device", StringType()))
    .add(StructField("Index", LongType()))
    .add(StructField("Model", StringType()))
    .add(StructField("User", StringType()))
    .add(StructField("gt", StringType()))
    .add(StructField("x", DoubleType()))
    .add(StructField("y", DoubleType()))
    .add(StructField("z", DoubleType()))
)
activityPath = f"{work_data}/activity-data"
customReader = (
    spark
    .readStream
    .format("json")
    .schema(customSchema)
    .option("maxFilesPerTrigger", 1)
    .load(activityPath)
)
customCounter = (
    customReader.groupBy("gt").count()
)
queryName = "custom_count"
customWriter = (
    customCounter
    .writeStream
    .queryName(queryName)
    .format("console")
    .outputMode("update")
)
checkpointLocation = f"{work_dir}/tmp/{queryName}"
!rm -rf $checkpointLocation
customTrigger = (
    customWriter
    .trigger(processingTime="1 second")
    .option("checkpointLocation", checkpointLocation)
)
customQuery = customTrigger.start()
customQuery.awaitTermination(10)
customQuery.stop()
```

</details>

In [67]:
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
customSchema = (
    StructType()
    .add(StructField("Arrival_Time", LongType()))
    .add(StructField("Creation_Time", LongType()))
    .add(StructField("Device", StringType()))
    .add(StructField("Index", LongType()))
    .add(StructField("Model", StringType()))
    .add(StructField("User", StringType()))
    .add(StructField("gt", StringType()))
    .add(StructField("x", DoubleType()))
    .add(StructField("y", DoubleType()))
    .add(StructField("z", DoubleType()))
)
activityPath = f"{work_data}/activity-data"
customReader = (
    spark
    .readStream
    .format("json")
    .schema(customSchema)
    .option("maxFilesPerTrigger", 1)
    .load(activityPath)
)
customCounter = (
    customReader.groupBy("gt").count()
)
queryName = "custom_count"
customWriter = (
    customCounter
    .writeStream
    .queryName(queryName)
    .format("console")
    .outputMode("update")
)
checkpointLocation = f"{work_dir}/tmp/{queryName}"
!rm -rf $checkpointLocation
customTrigger = (
    customWriter
    .trigger(processingTime="1 second")
    .option("checkpointLocation", checkpointLocation)
)
customQuery = customTrigger.start()
customQuery.awaitTermination(10)
customQuery.stop()

### <font color=red>3. [고급]</font> 스트리밍 처리가 완료된 이후에 파일이 추가되면 어떻게 동작하나요?

* 80초 이상 대기하여 카운트가 수렴한 시점에 data 경로 아래의 appended-1.json 파일을 수동으로 복사해 주고, 카운트에 변화가 있는지 확인합니다
  - 타임아웃 : 180초 이상 (애플리케이션이 파일을 복사할 시간을 충분히 가질 수 있도록)
  - 데이터 경로 : /home/jovyan/work/activity-data
  - JSON 파일("part-00000-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json")을 임의의 파일(appended-1.json")로 변경하여 복사합니다

<details><summary>[정답] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다

```python
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
! rm -rf /home/jovyan/work/lgde-spark-stream/tmp/activity_count
activityQuery = activityWriter.start()
activityQuery.awaitTermination(180)
activityQuery.stop()
    
# 80초 이후 카운트가 수렴되었을 때에 json 파일 추가 시에 아래와 같이 카운트 수가 증가합니다
notebook     | -------------------------------------------
notebook     | Batch: 80
notebook     | -------------------------------------------
notebook     | +----------+-------+
notebook     | |        gt|  count|
notebook     | +----------+-------+
notebook     | |       sit| 997023|
notebook     | |     stand| 922167|
notebook     | |stairsdown| 758424|
notebook     | |      walk|1073658|
notebook     | |      null| 846174|
notebook     | |  stairsup| 847050|
notebook     | |      bike| 874506|
notebook     | +----------+-------+
```

</details>

In [38]:
! rm -rf /home/jovyan/work/lgde-spark-stream/tmp/activity_count
activityQuery = activityWriter.start()
activityQuery.awaitTermination(180)
activityQuery.stop()

# docker
# $ cp /home/jovyan/work/data/activity-data/part-00000-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json /home/jovyan/work/data/activity-data/appended-1.json

### <font color=red>4. [중급]</font> 애플리케이션이 종료된 이후에 파일이 추가된 경우, 애플리케이션을 재기동하면 어떻게 동작하나요?

* 이전 예제에서 이미 애플리케이션이 종료었기 때문에, data 경로에 임의의 json 파일을 appended-2.json 으로| 수동으로 복사해 주고, 애플리케이션을 재기동하여 카운트에 변화가 있는지 확인합니다
  - 타임아웃 : 10초 이상 (기동 이후에 확인할 정도의 시간)
  - 데이터 경로 : /home/jovyan/work/activity-data
  - JSON 파일("part-00000-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json")을 임의의 파일(appended-2.json")로 변경하여 복사합니다

<details><summary>[정답] 출력 결과 확인 </summary>

> 아래와 유사하게 방식으로 작성 되었다면 정답입니다. 파일 복사는 수동으로 진행해도 무관합니다

```python
# 아래에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
!cp "$work_data/activity-data/part-00000-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json" "$work_data/activity-data/appended-2.json"
activityQuery = activityWriter.start()
activityQuery.awaitTermination(10)
activityQuery.stop()
    
# 애플리케이션이 재기동 되었을 때에 추가된 json 파일 크기만큼 아래와 같이 카운트 수가 증가합니다
notebook     | -------------------------------------------
notebook     | Batch: 81
notebook     | -------------------------------------------
notebook     | +----------+-------+
notebook     | |        gt|  count|
notebook     | +----------+-------+
notebook     | |       sit|1009332|
notebook     | |     stand| 933551|
notebook     | |stairsdown| 767789|
notebook     | |      walk|1086914|
notebook     | |      null| 856623|
notebook     | |  stairsup| 857502|
notebook     | |      bike| 885302|
notebook     | +----------+-------+
```

</details>

In [32]:
# 기동
!cp "$work_data/activity-data/part-00000-tid-730451297822678341-1dda7027-2071-4d73-a0e2-7fb6a91e1d1f-0-c000.json" "$work_data/activity-data/appended-2.json"
activityQuery = activityWriter.start()
activityStatus = activityQuery.status
activityQuery.awaitTermination(10) # 노트북 특성상, 대기하면 다음 실행을 할 수 없으므로 10초만 수행하고 종료합니다
activityQuery.stop()